In [ ]:
from classy import Class
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from scipy.special import roots_legendre
from numba import njit, prange
from SSLimPy.utils import utils
import os

In [ ]:
os.environ["MPI_NUM_THREADS"] = "14"

In [ ]:
@njit("(float64[::1], float64[::1], float64[::1], float64[:], float64[:])", parallel=True)
def convolve(kinter, wi, xi, Tk1, Tk2):
    kl = len(kinter)
    mu = xi
    Tkn = np.empty(kl)
    for ik in prange(kl):
        intgr_k1 = np.empty(kl)
        for ik1 in range(kl):
            q = np.sqrt(kinter[ik]**2 + kinter[ik1]**2 - 2 * kinter[ik] * kinter[ik1] * mu)
            Tq = -np.exp(utils.linear_interpolate(np.log(kinter), np.log(-Tk1), q))
            intgr_k1[ik1] = np.sum(wi * Tq)
        Tkn[ik] = np.trapz(2* np.pi * kinter**3 * intgr_k1 * Tk2, np.log(kinter))
    return Tkn

In [ ]:
xi, wi = roots_legendre(12)

In [ ]:
# fixed params
h = 0.65
Omegab = 0.05
Omegam = 0.3

g = 3.044/3
N_ur = 3.044 - g
T_ncdm = (4 / 11)**(1 / 3) * g**(1 / 4)
def compute_fixed(mnu):
    Omeganu = mnu * g**(0.75) / 94.07 / h**2
    Omegac = Omegam - Omegab - Omeganu
    return Omeganu, Omegac

cosmodict = {
    "output":"mPk, mTk",
    'P_k_max_1/Mpc': 50,
    'non linear' : 'hmcode2020',
    'z_max_pk' : 3.0,
    "N_ncdm": 1,
    "N_ur": N_ur,
    "h": h,
    "Omega_b": Omegab,
    "T_ncdm": T_ncdm
}

In [ ]:
def get_Transfers(mnu):
    cosmo = Class()

    cd = copy(cosmodict)
    Onu, Oc = compute_fixed(mnu)
    cd["Omega_ncdm"] = Onu
    cd["Omega_cdm"] = Oc

    cosmo.set(cd)
    cosmo.compute()
    mTk, k, z = cosmo.get_transfer_and_k_and_z()


    # Obtain cb Transfers
    fnu = cosmo.Omega_nu / cosmo.Omega0_m()
    fb = cosmo.Omega_b()/ cosmo.Omega_m()
    fc = cosmo.Omega0_cdm() / cosmo.Omega0_m()
    fcb = 1-fnu

    Tcb = (1 / fcb * (fb * mTk["d_b"] + fc * mTk["d_cdm"]))[:,-1]
    Tnu = mTk["d_ncdm[0]"][:,-1]
    Tmm = fcb * Tcb + fnu * Tnu

    kmask = k>0
    k = k[kmask]
    Tcb = Tcb[kmask]
    Tmm = Tmm[kmask]

    T2cc = convolve(k, wi, xi, Tcb, Tcb)
    T2mm = convolve(k, wi, xi, Tmm, Tmm)
    return k, Tcb, Tmm, T2cc, T2mm

In [ ]:
k,Tcc_low, Tmm_low, T2cc_low, T2mm_low = get_Transfers(0.06)
_,Tcc_high, Tmm_high, T2cc_high, T2mm_high = get_Transfers(0.12)

In [ ]:
plt.loglog(k, -Tcc_high)
plt.loglog(k, k**2)

In [ ]:
plt.semilogx(k, Tcc_high/Tcc_low -1)
plt.semilogx(k, Tmm_high/Tmm_low -1)

In [ ]:
response_cc = T2cc_low/T2cc_high -1
response_mm = T2mm_low/T2mm_high -1

In [ ]:
plt.semilogx(k, response_cc, label="clustering matter")
plt.semilogx(k, response_mm, label="total matter")
plt.xlabel(r"$k\; [\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$(T*T)(m_\nu=0.06\mathrm{eV})\:/\:(T*T)(0.12\mathrm{eV})-1$")
plt.legend()

In [ ]:
plt.loglog(k, T2cc_high, label="clustering matter")
plt.loglog(k, T2mm_high, label="total matter")
plt.xlabel(r"$k\; [\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$(T*T)(k)\;[\mathrm{Mpc}^{-3}]$")
plt.legend()

In [ ]:
T2mm_high/T2cc_high